<a href="https://www.kaggle.com/code/dialvedu/pneumonia-detection?scriptVersionId=99474288" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Pneumonia Detection in Chest X-Ray Images

By: Diego Vergara BSc

## Problem description

One of the most important aspects of healthcare, is a timely diagnosis of a disease, as it's easier to treat in early stages. This is not only appliable on cancer, but also in infections, along with other diseases.

Pneumonia is swelling or inflammation of the tissue in one or both lungs. It's usually caused by a bacterial infection, but it can also be caused by a virus. The symptoms of pneumonia can develop suddenly over 24 to 48 hours, or they may come on more slowly over several days. Common symptoms of pneumonia include cough, difficulty breathing, rapid heartbeat, high temperature, sweating, shivering and chest pain, among others. This disease can lead to death, so a prompt diagnosis can increase the chance of fast recovering and survival of the individual.

## Objective

Pneumonia is not easly identified in chest X-ray images, so the objective of this project is to create a model, that could be able to identify from images if a patient has pneumonia or not.

## Data

### Description

Chest X-ray images (anterior-posterior) were selected from retrospective cohorts of pediatric patients of one to five years old from Guangzhou Women and Children’s Medical Center, Guangzhou. All chest X-ray imaging was performed as part of patients’ routine clinical care.

For the analysis of chest x-ray images, all chest radiographs were initially screened for quality control by removing all low quality or unreadable scans. The diagnoses for the images were then graded by two expert physicians before being cleared for training the AI system. In order to account for any grading errors, the evaluation set was also checked by a third expert.

### Dataset

The dataset was obtained from the Kaggle dataset [Chest X-Ray Images (Pneumonia)](https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia), which comes from [Labeled Optical Coherence Tomography (OCT) and Chest X-Ray Images for Classification](https://data.mendeley.com/datasets/rscbjbr9sj/2) from Mendeley Data. According to de description of this dataset, data is organized into 3 folders (train, test, val) and contains subfolders for each image category (Pneumonia or Normal). There are 5863 X-Ray images (JPEG) and 2 categories according to de diagnosis (Pneumonia or Normal).

## Set up

In [ ]:
import pathlib
import os
import re
import warnings
import eli5
import shap

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from math import ceil
from matplotlib import gridspec
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, plot_roc_curve, plot_confusion_matrix
from IPython.display import Image, display

%matplotlib inline

In [ ]:
warnings.filterwarnings("ignore")

train_path = '../input/chest-xray-pneumonia/chest_xray/train'
test_path = '../input/chest-xray-pneumonia/chest_xray/test'
valid_path = '../input/chest-xray-pneumonia/chest_xray/val'

## Exploratory Data Analysis

EDA for images is different from what we can do with structured, text or time series data, so I took into account some [recomendations](https://towardsdatascience.com/exploratory-data-analysis-ideas-for-image-classification-d3fc6bbfb2d2) in this matter. The objective of this EDA is to identify differences between images.

### Image Visualization

Firstly, we are going to visualize three random images from each class in order to assess any difference, with a non-medical eye clearly.

In [ ]:
normal_imgs = [x for x in os.listdir(f'{train_path}/NORMAL') if x.endswith('.jpeg')]
pneumo_imgs = [y for y in os.listdir(f'{train_path}/PNEUMONIA') if y.endswith('.jpeg')]

select_norm = np.random.choice(normal_imgs, 3, replace = False)
select_pneu = np.random.choice(pneumo_imgs, 3, replace = False)

fig = plt.figure(figsize = (8,6))

for i in range(6):
    if i < 3:
        im_path = f'{train_path}/NORMAL/{select_norm[i]}'
        label = 'NORMAL'
    else:
        im_path = f'{train_path}/PNEUMONIA/{select_pneu[i-3]}'
        label = 'PNEUMONIA'
    
    ax = fig.add_subplot(2, 3, i+1)
    
    im = tf.keras.preprocessing.image.load_img(im_path, target_size = (100,100), color_mode='grayscale')
    plt.imshow(im, cmap='Greys_r')
    plt.title(label)
    plt.axis('off')

plt.show()

print(f'Number of normal images: {len(normal_imgs)}')
print(f'Number of pneumonia images: {len(pneumo_imgs)}')

In a first glance, we can se that in Pneumonia images, there is more white spots and noise in the lung area. Also, we have more Pneumonia images than Normal images, so we have a class imbalance that we need to solve.

For the next steps in the analysis, we are going to convert the images in a Numpy Array.

In [ ]:
size = (64, 64)

def imgs2np(path, file_list, size = (64, 64)):
    
    for i in file_list:
        image_path = path + i
        image = tf.keras.preprocessing.image.load_img(image_path, target_size = size, color_mode='grayscale')
        image = tf.keras.preprocessing.image.img_to_array(image)
        img_ts = [image.ravel()]
        
        try:
            np_array = np.concatenate((np_array, img_ts))

        except NameError:
            np_array = img_ts    

    return np_array

np_normal_images = imgs2np(f'{train_path}/NORMAL/', normal_imgs)
np_pneumonia_images = imgs2np(f'{train_path}/PNEUMONIA/', pneumo_imgs)

Then, we are going to obtain the mean image and the standard deviation image of both classes.

In [ ]:
mean_img_norm = np.mean(np_normal_images, axis=0).reshape(size)
mean_img_pneum = np.mean(np_pneumonia_images, axis=0).reshape(size)
std_img_norm = np.std(np_normal_images, axis=0).reshape(size)
std_img_pneum = np.std(np_pneumonia_images, axis=0).reshape(size)

ax = plt.subplot(2, 2, 1)
plt.imshow(mean_img_norm, cmap='Greys_r')
plt.title('Average Normal', fontsize=10)
plt.axis('off')

ax = plt.subplot(2, 2, 2)
plt.imshow(std_img_norm, cmap='Greys_r')
plt.title('Standard Deviation Normal', fontsize=10)
plt.axis('off')

ax = plt.subplot(2, 2, 3)
plt.imshow(mean_img_pneum, cmap='Greys_r')
plt.title('Average Pneumonia', fontsize=10)
plt.axis('off')

ax = plt.subplot(2, 2, 4)
plt.imshow(std_img_pneum, cmap='Greys_r')
plt.title('Standard Deviation Pneumonia', fontsize=10)
plt.axis('off')

plt.show()

We can see from the average images, that in patients without pneumonia the lungs are more clearly defined that those with pneumonia. There is also a higher standard deviation between pneumonia images, making clear the lung damage caused by the disease.

### Contrast Between Images

Now, I am going to compare the contrast between mean images, to get information about their differences.

In [ ]:
contrast_mean = mean_img_norm - mean_img_pneum
plt.imshow(contrast_mean, cmap='viridis')
plt.title(f'Difference Between Normal and Pneumonia Average')
plt.axis('off')
plt.colorbar()
plt.show()

In the image we can se that there is a negative difference in the lung area, showing there is a higher contrast in this area for pneumonia images.

### Eigenimages

Finally, I am going to analyse the eigenimages from the dataset, with a number of components enough to explain the 70% of the variance in the images.

In [ ]:
def eigenimages(im, n_comp = 0.7, size = (64, 64)):
    pca = PCA(n_components = n_comp, whiten = True)
    pca.fit(im)
    print('Number of PC: ', pca.n_components_)
    return pca
  
def plot_pca(pca, title, size = (64, 64)):
    n_comp = pca.n_components_
    fig = plt.figure(figsize=(8, 8))
    rows = int(n_comp**0.5)
    cols = ceil(n_comp/rows)
    for i in range(n_comp):
        ax = fig.add_subplot(rows, cols, i + 1, xticks = [], yticks = [])
        ax.imshow(pca.components_[i].reshape(size), 
                  cmap='Greys_r')
    plt.axis('off')
    fig.suptitle(title)
    plt.show()
    
plot_pca(eigenimages(np_normal_images), 'NORMAL')
plot_pca(eigenimages(np_pneumonia_images), 'PNEUMONIA')

From the eigenimages is clear that in Normal patients, the lung area has a well defined contour and rib cage, and in Pneumonia images this area is more diffuse.

## Dataset preparation

Now that we have information about how differentiable are images between them, we can say that a CNN model will have no issues classifying them. Taking this into account, we can prepare the datasets to train and validate the model.

In [ ]:
ds_train = image_dataset_from_directory(
    train_path,
    labels='inferred',
    label_mode='binary',
    image_size=(224,224),
    batch_size=32,
    shuffle=True,
)

ds_test = image_dataset_from_directory(
    test_path,
    labels='inferred',
    label_mode='binary',
    image_size=(224,224),
    batch_size=32,
    shuffle=False,
)

ds_valid = image_dataset_from_directory(
    valid_path,
    labels='inferred',
    label_mode='binary',
    image_size=(224,224),
    batch_size=1,
    shuffle=True,
)

I am going to convert the image data to float to get better performance from the model.

In [ ]:
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE

ds_train = ds_train.map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE)
ds_test = ds_test.map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE)
ds_valid = ds_valid.map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE)

## Model

One possible approach to this problem is convert the images to structured data, extrating relevant information like contrast, brightness, Fourier transform, among others, but an approach with CNN can help us extract relevant features related to pixel positions, which is what we are seeking when we want to get data from specific areas in the image, and the relationships between pixels in it. Taking this into account, I am going to use TensorFlow to build the classification model.

### Training hyperparameters

As we have a binary classification model, I can define some binary metrics like those from the confussion matrix, and the metrics that derive from it (AUC, Precission, Recall and PR ratio).

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.BinaryCrossentropy()
metrics = [tf.keras.metrics.BinaryAccuracy(name='binary_accuracy'),
           tf.keras.metrics.TruePositives(name='TP'),
           tf.keras.metrics.FalsePositives(name='FP'),
           tf.keras.metrics.TrueNegatives(name='TN'),
           tf.keras.metrics.FalseNegatives(name='FN'),
           tf.keras.metrics.Precision(name='P'),
           tf.keras.metrics.Recall(name='R'),
           tf.keras.metrics.AUC(name='AUC'),
           tf.keras.metrics.AUC(name='PR', curve='PR')
          ]

### Building a CNN

Currently, we have available a lot of pretrained models to extract better insights from image data. But first, the best is to build a CNN to check if a simpler model is enough for this problem. Also, we are going to add some regularization to the convolutional layers, to prevent any overfitting.

In [ ]:
regularizer = tf.keras.regularizers.L1L2(l1=0.01, l2=0.01)

model_c = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(224, 224, 3)),
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.Conv2D(8, 3, padding='same', activation='relu', kernel_regularizer=regularizer),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu', kernel_regularizer=regularizer),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', kernel_regularizer=regularizer),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu', kernel_regularizer=regularizer),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu', kernel_regularizer=regularizer),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(16, activation='relu', kernel_regularizer=regularizer),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_c.summary()

In [ ]:
model_c.compile(optimizer=optimizer, loss=loss, metrics=metrics)

history_c = model_c.fit(ds_train,
                        validation_data=ds_test,
                        epochs=50,
                       )

In [ ]:
history_frame_c = pd.DataFrame(history_c.history)
history_frame_c.loc[:, ['loss', 'val_loss']].plot()
history_frame_c.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot()

### Transfer learning - VGG16

In [ ]:
inputs = tf.keras.Input(shape=(224,224,3))
net = tf.keras.layers.Rescaling(1./255)(inputs)
net = tf.keras.layers.RandomFlip('horizontal')(net)

pretrained_base = tf.keras.applications.vgg16.VGG16(include_top=False,
                                                    pooling='avg',
                                                    input_tensor=net
                                                   )
pretrained_base.trainable = False

net = tf.keras.layers.Dropout(0.2)(pretrained_base.output)
net = tf.keras.layers.Dense(16, activation='relu')(net)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(net)

model_t = tf.keras.Model(inputs, outputs)

model_t.summary()

In [ ]:
model_t.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
history_t = model_t.fit(ds_train,
                        validation_data=ds_test,
                        epochs=50,
                       )

In [ ]:
history_frame_t = pd.DataFrame(history_t.history)
history_frame_t.loc[:, ['loss', 'val_loss']].plot()
history_frame_t.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot()

In [ ]:
model_t.evaluate(ds_valid, verbose=True)

### Training VGG 16

In [ ]:
inputs = tf.keras.Input(shape=(224,224,3))
net = tf.keras.layers.Rescaling(1./255)(inputs)
net = tf.keras.layers.RandomFlip('horizontal')(net)

pretrained_base = tf.keras.applications.vgg16.VGG16(include_top=False,
                                                    pooling='avg',
                                                    input_tensor=net
                                                   )
pretrained_base.trainable = True

net = tf.keras.layers.Dropout(0.2)(pretrained_base.output)
net = tf.keras.layers.Dense(16, activation='relu')(net)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(net)

model_t_2 = tf.keras.Model(inputs, outputs)

model_t_2.summary()

In [ ]:
model_t_2.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
history_t_2 = model_t_2.fit(ds_train,
                            validation_data=ds_test,
                            epochs=50,
                           )

In [ ]:
history_frame_t_2 = pd.DataFrame(history_t_2.history)
history_frame_t_2.loc[:, ['loss', 'val_loss']].plot()
history_frame_t_2.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot()

In [ ]:
model_t_2.evaluate(ds_valid, verbose=True)

### Transfer learning - VGG16 Max pool

In [ ]:
inputs = tf.keras.Input(shape=(224,224,3))
net = tf.keras.layers.Rescaling(1./255)(inputs)
net = tf.keras.layers.RandomFlip('horizontal')(net)

pretrained_base = tf.keras.applications.vgg16.VGG16(include_top=False,
                                                    pooling='max',
                                                    input_tensor=net
                                                   )
pretrained_base.trainable = False

net = tf.keras.layers.Dropout(0.2)(pretrained_base.output)
net = tf.keras.layers.Dense(16, activation='relu')(net)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(net)

model_t_3 = tf.keras.Model(inputs, outputs)

model_t_3.summary()

In [ ]:
model_t_3.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
history_t_3 = model_t_3.fit(ds_train,
                            validation_data=ds_test,
                            epochs=50,
                           )

In [ ]:
history_frame_t_3 = pd.DataFrame(history_t_3.history)
history_frame_t_3.loc[:, ['loss', 'val_loss']].plot()
history_frame_t_3.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot()

In [ ]:
model_t_3.evaluate(ds_valid, verbose=True)

### Transfer learing VGG16 - Avg Pool, Regularization

In [ ]:
regularizer = tf.keras.regularizers.L1L2(l2=0.01)

inputs = tf.keras.Input(shape=(224,224,3))
net = tf.keras.layers.Rescaling(1./255)(inputs)
net = tf.keras.layers.RandomFlip('horizontal')(net)

pretrained_base = tf.keras.applications.vgg16.VGG16(include_top=False,
                                                    pooling='avg',
                                                    input_tensor=net
                                                   )
pretrained_base.trainable = False

net = tf.keras.layers.Dropout(0.2)(pretrained_base.output)
net = tf.keras.layers.Dense(16, activation='relu', kernel_regularizer=regularizer)(net)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(net)

model_t_4 = tf.keras.Model(inputs, outputs)

model_t_4.summary()

In [ ]:
model_t_4.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
history_t_4 = model_t_4.fit(ds_train,
                            validation_data=ds_test,
                            epochs=50,
                           )

In [ ]:
history_frame_t_4 = pd.DataFrame(history_t_4.history)
history_frame_t_4.loc[:, ['loss', 'val_loss']].plot()
history_frame_t_4.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot()

In [ ]:
model_t_4.evaluate(ds_valid, verbose=True)

## Visualization and Explainability

In [ ]:
def get_img_array(img_path, size):
    
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=size)
    array = tf.keras.preprocessing.image.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        model.inputs,
        [model.get_layer(last_conv_layer_name).output,
         model.output
        ]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        p = grad_model(img_array)
        last_conv_layer_output = p[0]
        preds = p[1]
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def display_gradcam(img_path, heatmap, size, cam_path="cam.jpg", alpha=0.4):
    # Load the original image
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=size)
    img = tf.keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)
    
    # Display Grad-CAM
    
    plt.imshow(superimposed_img)
    plt.axis('off')

In [ ]:
normal_imgs_val = [x for x in os.listdir(f'{valid_path}/NORMAL') if x.endswith('.jpeg')]
pneumo_imgs_val = [y for y in os.listdir(f'{valid_path}/PNEUMONIA') if y.endswith('.jpeg')]

select_norm_val = np.random.choice(normal_imgs_val, 1, replace = False)
select_pneu_val = np.random.choice(pneumo_imgs_val, 1, replace = False)

path = f'{valid_path}/NORMAL/{select_norm_val[0]}'

img_array = get_img_array(path, size=(224, 224, 3))

heatmap = make_gradcam_heatmap(img_array, model_t_4, 'block5_conv3')

In [ ]:
display_gradcam(path, heatmap, size=(224,224))
plt.title('Normal')

In [ ]:
path = f'{valid_path}/PNEUMONIA/{select_pneu_val[0]}'

img_array = get_img_array(path, size=(224, 224, 3))

heatmap = make_gradcam_heatmap(img_array, model_t_4, 'block5_conv3')

In [ ]:
display_gradcam(path, heatmap, size=(224,224))
plt.title('Pneumonia')

# References